Download dependencies:

In [5]:
!pip install zipp
!pip install requests
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


Import Dependencies:

In [6]:
from zipfile import ZipFile
import requests
import pandas as pd

Download the file:

In [5]:
url = 'https://download-data.deutschebahn.com/static/datasets/wifi/20171212_wifionice.zip'
response = requests.get(url)

with open('20171212_wifionice.zip', 'wb') as f:
    f.write(response.content)

Extract the csv file and read dataset:

In [3]:
with ZipFile('20171212_wifionice.zip', 'r') as zip_file:
    zip_file.extract('surveyor_hackathon_data_20171212.csv')

data = pd.read_csv('surveyor_hackathon_data_20171212.csv', sep=';', low_memory=False)